<a href="https://colab.research.google.com/github/Sakippo/IntroToMachineLearning/blob/main/GoogLeNet_for_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備
必要パッケージのインストール

In [ ]:
! pip install torchinfo
!pip install d2l==0.17.2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchinfo import summary
from torch import Tensor
from typing import Optional, Tuple, List, Callable, Any
from d2l import torch as d2l

# Fashion MNISTデータセットのダウンロード

In [ ]:
train_loader, test_loader = d2l.load_data_fashion_mnist(256, (56, 56))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# モデルの定義




Inception Module

In [ ]:
class Inception(nn.Module):
      def __init__(self,imput: int,filters_1x1: int, filters_3x3_reduce: int,filters_3x3: int,filters_5x5_reduce: int,filters_5x5: int,filters_pool: int):
        super().__init__()
        
        self.path1_1=  nn.Conv2d(imput, filters_1x1, kernel_size=1, stride=1)

        self.path2_1 = nn.Conv2d(imput,filters_3x3_reduce, kernel_size=1, stride=1)
        self.path2_2 = nn.Conv2d(filters_3x3_reduce, filters_3x3, kernel_size=3, stride=1,padding=1)
        

        self.path3_1 = nn.Conv2d(imput, filters_5x5_reduce, kernel_size=1, stride=1)
        self.path3_2 = nn.Conv2d(filters_5x5_reduce,filters_5x5, kernel_size=5, stride=1,padding=2)
        

        self.path4_1 = nn.MaxPool2d(3, stride=1, padding=1)
        self.path4_2 = nn.Conv2d(imput,filters_pool, kernel_size=1, stride=1)
      
      def forward(self, x: Tensor) -> List[Tensor]:
          path1 = self.path1_1(x)
          path1 = F.relu(path1)

          path2 = self.path2_1(x)
          path2 = F.relu(path2)

          path2 = self.path2_2(path2)
          path2 = F.relu(path2)

          path3 = self.path3_1(x)
          path3 = F.relu(path3)

          path3 = self.path3_2(path3)
          path3 = F.relu(path3)

          path4 = self.path4_1(x)
          path4 = self.path4_2(path4)
          path4 = F.relu(path4)

          return torch.concat([path1, path2, path3, path4], axis=1)

Auxiliary Classifier

In [ ]:
class Aux(nn.Module):
    def __init__(self,imput: int):
        super().__init__() 

        self.conv = nn.Conv2d(imput, 128, kernel_size = 1, stride=1) 
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 10)
        self.dropout = nn.Dropout(0.7)

    def forward(self, x: Tensor) -> List[Tensor]:
        x = F.adaptive_avg_pool2d(x, (4, 4))
        x = self.conv(x) 
        x = F.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x) 
        x = self.dropout(x)
        x = self.fc2(x) 

        return x

In [ ]:
class MyMnistNet(nn.Module):
    def __init__(self, blocks: Optional[List[Callable[..., nn.Module]]] = None):
        super().__init__()
        blocks = [Inception,Aux]

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2,padding=3) 
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1, stride=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.45)
        self.fc1 = nn.Linear(1024, 10)
        
        self.batchnorm = nn.BatchNorm2d(1024)

        self.inception3a = Inception(192,64,96,128,16,32,32);
        self.inception3b = Inception(256,128,128,192,32,96,64);

        self.inception4a = Inception(480,192,96,208,16,48,64);
        self.inception4b = Inception(512,160,112,224,24,64,64);
        self.inception4c = Inception(512,128,128,256,32,64,64);
        self.inception4d = Inception(512,112,144,288,32,64,64);
        self.inception4e = Inception(528,256,160,320,32,128,128);

        self.inception5a = Inception(832,256,160,320,32,128,128);
        self.inception5b = Inception(832,384,192,384,48,128,128);

        self.aux1 = Aux(512);
        self.aux2 = Aux(528);

        
    def forward(self, x):
      
        x = self.conv1(x)  # 畳み込み層
        x = F.relu(x)
        
      
        x = nn.MaxPool2d(2)(x) # Max Pooling
       
        x = self.conv2(x)  # 畳み込み層
        x = F.relu(x)
        
        
        x = self.conv3(x)  # 畳み込み層
        x = F.relu(x)
        
        x = nn.MaxPool2d(2)(x)  # Max Pooling
       
        # Inception Module 3a,3b
        x= self.inception3a(x)  
        x= self.inception3b(x)
        
        x = nn.MaxPool2d(2)(x) # Max Pooling
        
        # Inception Module 4a
        x= self.inception4a(x)

        # Auxiliary Classifier1
        aux1: Optional[Tensor] = None
        aux1= self.aux1(x)

        # Inception Module 4b,4c,4d
        x= self.inception4b(x)
        x= self.inception4c(x)
        x= self.inception4d(x)

        # Auxiliary Classifier2
        aux2: Optional[Tensor] = None
        aux2= self.aux2(x)

        # Inception Module 4e
        x= self.inception4e(x)
       
        x = nn.MaxPool2d(2)(x) # Max Pooling
        
        # Inception Module 5a,5e
        x= self.inception5a(x)
        x= self.inception5b(x)
  
        x = self.batchnorm(x) #Batch Normalization

        x = self.avgpool(x) # Average Pooling
        x = torch.flatten(x, 1)
        
        x = self.dropout(x) # Dropout
        
        x = self.fc1(x)  #Linear層

        return x, aux1,aux2 

      # 各層の出力サイズを確認
print( summary(MyMnistNet(), input_size=(64, 1, 56, 56)) )

Layer (type:depth-idx)                   Output Shape              Param #
MyMnistNet                               --                        --
├─Conv2d: 1-1                            [64, 64, 28, 28]          3,200
├─Conv2d: 1-2                            [64, 64, 14, 14]          4,160
├─Conv2d: 1-3                            [64, 192, 14, 14]         110,784
├─Inception: 1-4                         [64, 256, 7, 7]           --
│    └─Conv2d: 2-1                       [64, 64, 7, 7]            12,352
│    └─Conv2d: 2-2                       [64, 96, 7, 7]            18,528
│    └─Conv2d: 2-3                       [64, 128, 7, 7]           110,720
│    └─Conv2d: 2-4                       [64, 16, 7, 7]            3,088
│    └─Conv2d: 2-5                       [64, 32, 7, 7]            12,832
│    └─MaxPool2d: 2-6                    [64, 192, 7, 7]           --
│    └─Conv2d: 2-7                       [64, 32, 7, 7]            6,176
├─Inception: 1-5                         [64, 480, 

# 学習とテスト


In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    out = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        p_y_hat = model(x)[0]
        p_y_aux1 = model(x)[1]
        p_y_aux2 = model(x)[2]

        loss = F.cross_entropy(p_y_aux1*0.3+p_y_aux2*0.3+p_y_hat, y)  #loss = aux1 の損失 * 0.3 + aux2 の損失 * 0.3 + 最後の出力層の損失

        optimizer.zero_grad()
        loss.backward()  # 勾配計算
        optimizer.step()  # パラメータ更新
        if batch_idx  % 100 == 0:
            print(f"Epoch={epoch+1}, Batch={batch_idx+1:03}, Loss={loss.item():.4f}")
            out = loss.item()
    return out

def test(model, device, test_loader):
    model.eval()
    correct = 0
    for x, y in test_loader:
      x = x.to(device)
      y = y.to(device)
      p_y_hat = model(x)[0]

      y_hat = p_y_hat.argmax(dim=1, keepdim=True) 
      correct += y_hat.eq(y.view_as(y_hat)).sum().item()

    accuracy = correct / len(test_loader.dataset)
    print(f"Test-set accuracy={accuracy :.04f}\n")
    return accuracy

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MyMnistNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(14):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

main()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch=1, Batch=001, Loss=2.3044
Epoch=1, Batch=101, Loss=0.6750
Epoch=1, Batch=201, Loss=0.4064
Test-set accuracy=0.7237

Epoch=2, Batch=001, Loss=0.3869
Epoch=2, Batch=101, Loss=0.4241
Epoch=2, Batch=201, Loss=0.2854
Test-set accuracy=0.8615

Epoch=3, Batch=001, Loss=0.3027
Epoch=3, Batch=101, Loss=0.2897
Epoch=3, Batch=201, Loss=0.3193
Test-set accuracy=0.8749

Epoch=4, Batch=001, Loss=0.2830
Epoch=4, Batch=101, Loss=0.2614
Epoch=4, Batch=201, Loss=0.2974
Test-set accuracy=0.8482

Epoch=5, Batch=001, Loss=0.2185
Epoch=5, Batch=101, Loss=0.2511
Epoch=5, Batch=201, Loss=0.1936
Test-set accuracy=0.8495

Epoch=6, Batch=001, Loss=0.1921
Epoch=6, Batch=101, Loss=0.2050
Epoch=6, Batch=201, Loss=0.2116
Test-set accuracy=0.9028

Epoch=7, Batch=001, Loss=0.1908
Epoch=7, Batch=101, Loss=0.1721
Epoch=7, Batch=201, Loss=0.2046
Test-set accuracy=0.9087

Epoch=8, Batch=001, Loss=0.2171
Epoch=8, Batch=101, Loss=0.2203
Epoch=8, Batch=201, Loss=0.1820
Test-set accuracy=0.8994

Epoch=9, Batch=001, Loss